In [ ]:
import warnings; warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
import logging
import torch
from transformers import *
from pytorch_pretrained_bert import BertAdam
from utils.datasets import *
from utils.processors import *
from utils.processors_additional import *
from utils.train import *
from utils.datasets import BertDataset as BDS
#from utils import BertDataset, NERProcessor, InputExampleToTensors, NERTrainer,InputExample,SwedishNERCorpusProcessor,SUCProcessor
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
import nltk
import matplotlib.pyplot as plt

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

### 0. Settings

In [ ]:
#model_name = 'bert-base-uncased'
model_name = 'bert-base-multilingual-uncased'
#model_name = './swe-uncased_L-12_H-768_A-12'

In [ ]:
dataset_path = 'SUC/moreTags/'

### 1. Example

In [ ]:
example_sentence = \
    'iran har hittills inte reagerat på någondera av de stora påkarna som saudier och irakier hött med :' + \
    ' landsbergis ansåg att gorbatjovs lördagsappell visade att denne ignorerar vädjanden från väst om att' + \
    ' börja tala med regeringen i vilnius .'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
tokenized_text = tokenizer.tokenize(example_sentence)

In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
len(indexed_tokens), indexed_tokens[:3]

### 2. SUC Processor

In [ ]:
processor = SUCProcessor(dataset_path, tokenizer, do_lower_case=True)

In [ ]:
label_list = processor.get_label_list()
label_list

#### Prune Examples (Temp)

In [ ]:
def prune_examples(_examples, ratio=None):
    if ratio is None:
        return _examples
    else:
        num_examples_new = int(ratio*float(len(_examples)))
        print(f'use {num_examples_new} of {len(_examples)} examples')
        return _examples[:num_examples_new]

prune_examples([0, 1, 2, 3, 4, 5], ratio=0.5)

In [ ]:
prune_ratio = 0.02

#### Train Data

In [ ]:
train_examples_all = processor.get_train_examples()

In [ ]:
train_examples = prune_examples(train_examples_all, ratio=prune_ratio)

In [ ]:
print(train_examples[8].guid)
print(train_examples[8].text_a)
print(train_examples[8].text_b)
print(train_examples[8].label)

#### Validation Data

In [ ]:
val_examples_all = processor.get_test_examples()

In [ ]:
val_examples = prune_examples(val_examples_all, ratio=prune_ratio)

In [ ]:
print(len(val_examples))
print(val_examples[1].text_a)
print(val_examples[1].label)

#### Dataloader

In [ ]:
batch_size = 16

In [ ]:
#B-LOC O B-TME O O O O O O O O O O O O O O O B-PRS O O B-PRS O O O O O O O O O O O O O O O B-LOC O
samples_transformer = InputExampleToTensors(tokenizer, max_seq_length=64, label_list=label_list)

In [ ]:
train_data = BDS(train_examples, transform=samples_transformer)
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=batch_size)

In [ ]:
validation_data = BDS(val_examples, transform=samples_transformer)
valid_dataloader = DataLoader(validation_data, sampler=SequentialSampler(validation_data), batch_size=batch_size)

### 3. Train

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))
model

In [ ]:
trainer = NERTrainer(model, train_dataloader, valid_dataloader, label_list, fp16=True if device == "cuda" else False)
trainer

In [ ]:
trainer.fit(learning_rate=2e-5, num_epochs=1)  # 4

#### Different training rates

#### Save Model Checkpoint

### 4. Investigate

In [ ]:
import numpy as np

In [ ]:
sent_tokenizer = tokenizer
sentence_lower = 'En som arbetar mycket hårt är erik som är politisk aktiv inom anderst i sverige .'
sentence_lower = sentence_lower.lower()
print(sentence_lower)

In [ ]:
example = InputExample("", sentence_lower, label='O')
example

In [ ]:
to_tensors = InputExampleToTensors(tokenizer, max_seq_length=128, label_list=label_list)
input_ids, input_mask, segment_ids, label_id = to_tensors(example)
input_ids, input_mask, segment_ids, label_id

In [ ]:
tokens_tensor = input_ids.view(1,-1)
segments_tensors = segment_ids.view(1,-1)

In [ ]:
if device == 'cuda':
    model.to('cuda')
    tokens_tensor.to('cuda')
    segments_tensors.to('cuda')

In [ ]:
model.eval()

In [ ]:
logits = model(tokens_tensor, segments_tensors)
logits

In [ ]:
res = []
res.extend(logits[0].argmax(-1))
res

In [ ]:
np_logits = logits[0].detach().cpu().numpy()
np.argmax(np_logits, axis=2)

In [ ]:
lst = np.argmax(np_logits, axis=2)[0].tolist()
lst = lst[1:]
lst

In [ ]:
splitinput = tokenizer.tokenize(sentence_lower)
splitinput

In [ ]:
for num, word in zip(lst, splitinput):
    if num == 4:
        print("PERSON: " + word)
    elif num == 5:
        print("ORG: " + word)
    elif num == 6:
        print("LOCATION: " + word)
    elif num == 7:
        print("WORK: " + word)
    elif num == 8:
        print("PRODUCT: " + word)
    else:
        print(num, word)
#    if num == 4 or num == 5:
#        print('{} {}'.format(num, word))

In [ ]:
print("PER: " + str(trainer.total_per_correct / trainer.total_per))
print("LOC: " + str(trainer.total_loc_correct / trainer.total_loc))
print("ORG: " + str(trainer.total_org_correct / trainer.total_org))

In [ ]:
trainer.labelDict

In [ ]:
trainer.val_f1_score_hist

In [ ]:
def cluster(my_list, n):
    final = [my_list[i * n:(i + 1) * n] for i in range((len(my_list) + n - 1) // n )]
    return list(map(lambda x: sum(x)/len(x), final))

In [ ]:
import matplotlib.pyplot as plt
val = cluster(trainer.val_f1_score_hist, 50)
plt.plot(val)